In [1]:
import os, os.path
import sys
import time

import shutil

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:

import torch
from torch.nn import Linear
import torch.nn as nn
from torch_geometric.nn import GCNConv

# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

from torch.utils.data import random_split




In [4]:

sys.path.append('../../ai.rdee')
import air

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
class TGCN_dataset(torch.utils.data.Dataset):
    def __init__(self):
        for f in self.processed_file_names:
            if not os.path.exists(f):
                self.process()
        self.data = torch.load(self.processed_file_names[0])

    @property
    def raw_file_names(self):
        return ['data/sz_adj.csv', 'data/sz_speed.csv']

    @property
    def processed_file_names(self):
        return ['data_sz_GRU.pt']

    def download(self):
        # Download to `self.raw_dir`.
        print("Please download the data manually!")
        sys.exit(0)
        
    def process(self):
        for f in self.raw_file_names:
            if not os.path.exists(f):
                slef.download()
                break
        print("Processing ... ...")
        # Read data into huge `Data` list.
        fn_adj, fn_spd = self.raw_file_names
        df_spd = pd.read_csv(fn_spd)
        S = df_spd.values  # nTimes x nNodes
        
        data = np.zeros((S.shape[0]-14, 15, S.shape[1]))
        for i in range(data.shape[0]):
            data[i,:,:] = S[i:i+15, :]
        
        torch.save(torch.Tensor(data), self.processed_file_names[0])
        print("Process done.")
        
    def clean(self):
        os.remove('data_sz_GRU.pt')
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx: int):
        return self.data[idx, :, :]

In [7]:
ds = TGCN_dataset()

In [8]:
ratio_valid = 0.2

In [9]:
size_valid = int(ratio_valid * len(ds))
size_train = len(ds) - size_valid
ds_learn, ds_valid = random_split(ds, [size_train, size_valid])

In [10]:
def collate_fn(batch):
#     global b1
#     b1 = batch
    # batch: (batchSize, seq, nFeatures)
    batchTS = torch.stack(batch)
    return batchTS[:, :12, :].permute(0,2,1).contiguous(), batchTS[:, 12:, :].permute(0,2,1).contiguous()
#     return batchTS[:, :12, :], batchTS[:, 12:, :].permute(0, 2, 1).contiguous()
#     return batchTS[:, :12, :].permute(0,2,1).contiguous(), batchTS[:, 12:, :].permute(0,2,1).contiguous()# .view(batchTS.shape[0], -1)

In [11]:
dlr_train = torch.utils.data.DataLoader(ds_learn, shuffle=True, batch_size=64, collate_fn=collate_fn)

In [12]:
dlr_valid = torch.utils.data.DataLoader(ds_valid, shuffle=True, batch_size=size_valid, collate_fn=collate_fn)

In [13]:
a, b = next(iter(dlr_train))
print(a.shape)
print(b.shape)

torch.Size([64, 156, 12])
torch.Size([64, 156, 3])


In [14]:
class GRU(nn.Module):
    def __init__(self, nNodes, input_dim, GRU_dim, output_dim):
        super().__init__()
        self.GRULayer = nn.GRU(input_dim, GRU_dim, batch_first=True)
        self.Regressor = nn.Linear(GRU_dim, output_dim)
        
    def forward(self, inputs):
        batch_size, num_nodes, seq_len = inputs.shape
#         h0 = torch.zeros(batch_size, num_nodes * self.GRU_dim).type_as(
#             inputs
#         )
#         print(inputs.shape)
        out, h_gru = self.GRULayer(inputs.view(batch_size * num_nodes, seq_len, 1))  # h_gru : (1, batchsize, hidden_dim)
#         print(h_gru.shape)
        x = self.Regressor(h_gru.squeeze(0))
        return x.view(batch_size, num_nodes, -1)  # x(batch_size * num_nodes, output_dim)

In [15]:
net = GRU(156, 1, 100, 3)

In [16]:
net.to(device)

GRU(
  (GRULayer): GRU(1, 100, batch_first=True)
  (Regressor): Linear(in_features=100, out_features=3, bias=True)
)

In [17]:
optim = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1.5e-3)
criterion = nn.MSELoss()

esp = air.utils.EarlyStopping(30)



In [18]:
xV, yV = next(iter(dlr_valid))

In [19]:
lossesT = []
lossesE = []
lossesV = []

for epoch in range(30):
    lossesE = []
    net.train()
    for x, y in dlr_train:
        optim.zero_grad()

        y_hat = net(x.to(device))
        loss = criterion(y_hat.view(-1), y.view(-1).to(device))
        if np.isnan(loss.item()):
            print("nan loss! Stop")
            break
        loss.backward()
        optim.step()
        lossesE.append(loss.item())
    lossesT.append(np.array(lossesE).mean())
    
    net.eval()
    
    with torch.no_grad():
        yV_hat = net(xV.to(device))
        lossesV.append(criterion(yV_hat.view(-1), yV.view(-1).to(device)).item())
    if epoch % 10 == 0:
        print(f"epoch={epoch}, lossT={lossesT[-1]:.3f}, lossV={lossesV[-1]:.3f}, minV={esp.val_loss_min:.3f}  |  @{time.ctime()}")
    esp(lossesV[-1], net)
    if esp.early_stop:
        print(f"Early stopping at {epoch}")
        break

    

epoch=0, lossT=259.935, lossV=219.447, minV=inf  |  @Sun Oct 29 15:37:13 2023
epoch=10, lossT=53.477, lossV=55.680, minV=58.878  |  @Sun Oct 29 15:37:28 2023
epoch=20, lossT=38.602, lossV=40.913, minV=41.790  |  @Sun Oct 29 15:37:44 2023
